In [1]:
import pandas as pd
import random
import os
import numpy as np
import string
import re
import pickle
import time
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import Imputer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import model_selection
from sklearn import metrics
from textblob import TextBlob, Word
from sklearn.externals import joblib
from scipy.stats import spearmanr, pearsonr
from sklearn.manifold import TSNE
%matplotlib inline
from bokeh.io import push_notebook, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LabelSet
import gensim
import splitter
from sklearn.linear_model import Ridge

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

/Users/rithika/anaconda3/envs/deep/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
import torch

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

In [4]:
#loading googlenews vec
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/rithika/Documents/247ai/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
#cleaning input data to get predictor values and target
def prepare_data(fname):
    filename = os.path.join('/Users/rithika/Documents/247ai/datasets', fname)
    if os.path.isfile(filename):
        dat = pd.read_csv(filename)
    else:
        print("no such file exists at this time")
    dat['Adcopy']=dat['Headline 1']+' '+dat['Headline 2']+' '+dat['Description']
    #dat['Adcopy'] = dat['Description']
    dat = dat[['Adcopy','Clicks']]
    dat = dat[pd.notnull(dat['Adcopy'])]
    dat['Adcopy'] = dat['Adcopy'].replace('http\S+|www.\S+', '', regex=True)
    dat['Adcopy'].replace('[™!®"#\'©()*+,-./:;<=>?@\&[\]^_`{|}~’”“′‘\\\%0123456789£]',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('  ',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('   ',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('  ',' ',inplace=True,regex=True)
    dat['Adcopy'] = dat['Adcopy'].str.lower()
    dat['Adcopy'] = dat['Adcopy'].apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(str(word)) for word in x.split()]))
    dat['Copy'] = dat['Adcopy'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    data = dat[['Copy','Clicks']]
    datum = data.groupby('Copy')['Clicks'].mean().reset_index()
    print(len(datum))
    return datum

In [6]:
#prints words from the dataset which is not present in google news vec(simply for reducing the runtime of preprocessing)
def no_present(da, model):
    #loading googlenews vec
    #model = gensim.models.KeyedVectors.load_word2vec_format('/Users/rithika/Documents/247ai/GoogleNews-vectors-negative300.bin', binary=True)
    nokey = list()
    for sentence in da:
        for word in sentence.split(' '): 
            if word not in model:
                if word not in nokey:
                    nokey.append(word)
    #print(nokey)
    #print(len(nokey))
    return nokey

In [7]:
#splits the compound words
def spli(row, nokey):
    if row in nokey:
        y = splitter.split(row) #compound word splitter
        if y != row and y != '':
            return ' '.join(y)
        else:
            wo = Word(row).correct() #spellcorrector
            return wo
    else:
        return row

In [8]:
#furthur cleans the data and returns the input and output values
def clean_dat(data,model):
    nokey = no_present(data['Copy'],model)
    data['Copy'] = data['Copy'].apply(lambda x: ' '.join([spli(str(word),nokey) for word in x.split()]))
    datu = data.groupby('Copy')['Clicks'].mean().reset_index()
    clean_d = datu['Copy'] #predictor values
    res = datu['Clicks'] #target
    print(len(clean_d))
    print(len(res))
    return clean_d, res, datu

In [9]:
# removes rows which have words from foreign language
def remove_lang(clean_d, datu, model):
    key_not = no_present(clean_d, model)
    ind = []
    count = 0
    for sen in clean_d:
        for word in sen.split(' '):
            if word in key_not:
                if word not in stop:
                    if count not in ind:
                        ind.append(count)
        count+=1
    cl_data = datu.drop(datu.index[ind])  
    cl_data['Copy'] = cl_data['Copy'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #print(ind)
    clean_data = cl_data['Copy']
    result = cl_data['Clicks']
    print(len(clean_data))
    print(len(result))
    return clean_data, result

In [10]:
def bow_tfidf(cleaned_data, result):
    cleaned_data = cleaned_data.tolist()
    # create the transform
    vectorizer = TfidfVectorizer()
    # tokenize and build vocab
    vec_train = vectorizer.fit_transform(cleaned_data)
    #Save vectorizer.vocabulary_
    pickle.dump(vectorizer,open("feature_bow.pkl","wb")) # used to predict unseen data
    # encode document
    #vector = vectorizer.transform(cleaned_data)
    X = np.array(vec_train.toarray())
    y = np.array(result)
    # summarize
    #print(vectorizer.vocabulary_)
    # summarize encoded vector
    #print(vec_train.shape)
    #print(vec_train.toarray())
    print(X.shape)
    print(y.shape)
    return X, y 

In [11]:
# a list of random float values within a range
def sample_floats(low, high, k=1):
    result = []
    seen = set()
    for i in range(k):
        x = random.uniform(low, high)
        while x in seen:
            x = random.uniform(low, high)
        seen.add(x)
        result.append(x)
    return result

In [12]:
#generates vector array for words not in googlenews vec
def gen_arr():
    c = [0] * 300
    dup = sample_floats(-0.001, 0.001, 300)
    r = np.random.randint(5, 30)
    for i in range(r):
        ind = np.random.randint(0, len(c) - 1)
        c[ind] = np.random.choice(dup)
    return c 

In [13]:
#getting the words not present in google model
def get_valvec(data):
    nokey = list()
    my_dict = {}
    for sentence in data:
        for word in sentence.split(' '): 
            if word not in model: 
                if word not in nokey: #for words not present    
                    nokey.append(word)
                    ne = gen_arr()
                    my_dict[word] = ne
    return my_dict

In [14]:
#getting feature vectors for hello fresh
def get_vectors(data):
    extra_vec = get_valvec(cleaned_data)     #dictionary of words not present in google with their vecs 
    in_arr = []
    for sentence in data:
        n_arr = []
        for word in sentence.split(' '): 
            arr = []
            try:
                for single in model[word]:
                    arr.append(single)
            except ValueError:
                print("Value error")
            except KeyError:
                n_w = word.title()
                nn_w = Word(word).correct()
                if n_w in model:
                    try:
                        for single in model[n_w]:
                            arr.append(single)
                    except ValueError:
                        print("Value error")
                elif nn_w in model:
                    try:
                        for single in model[nn_w]:
                            arr.append(single)
                    except ValueError:
                        print("Value error")
                else:
                    v = extra_vec.get(word)
                    for i in v:
                        arr.append(i)
            n_arr.append(arr)
            arry = []
            arry = np.sum(n_arr,axis=0)
        in_arr.append(arry)   
    return in_arr

In [15]:
def word2vec(cleaned_data, result):
    #cleaned_data = cleaned_data.astype(str)
    res = get_vectors(cleaned_data)
    x = np.array(res).tolist()
    X = np.array(x)
    y = np.array(result)
    print(X.shape)
    print(y.shape)
    return X, y

In [16]:
def g_uni_encoder(cleaned_data, result):
    cleaned_data = cleaned_data.tolist()
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
    #to suppress INFO prints by tensorflow
    tf.logging.set_verbosity(tf.logging.ERROR)
    # Import the Universal Sentence Encoder's TF Hub module
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(cleaned_data))
    #print(message_embeddings.shape)
    X = message_embeddings
    y = np.array(result)
    print(X.shape)
    print(y.shape)
    return X, y

In [17]:
def infer_se(cleaned_data, result):    
    from models import InferSent
    V = 2
    MODEL_PATH = '/Users/rithika/Documents/InferSent/infersent%s.pkl' % V
    params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                    'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
    infersent = InferSent(params_model)
    infersent.load_state_dict(torch.load(MODEL_PATH))
    W2V_PATH = '/Users/rithika/Documents/InferSent/crawl-300d-2M.vec'
    infersent.set_w2v_path(W2V_PATH)
    infersent.build_vocab(cleaned_data, tokenize=False)
    embeddings = infersent.encode(cleaned_data, tokenize=False) #True)
    print('nb sentences encoded : {0}'.format(len(embeddings)))
    X = embeddings
    y = np.array(result)
    print(X.shape)
    print(y.shape)
    return X, y

In [19]:
### Running the functions defined above

In [18]:
#normal preprocess
data = prepare_data('Subaru_2018-06-25_2018-07-24.csv')

145


In [19]:
#prints words from the dataset which is not present in google news vec
key1 = list()
for sentence in data['Copy']:
    for word in sentence.split(' '): 
        if word not in model:
            if word not in key1:
                key1.append(word)
                
print(len(key1))
print(key1)

10
['customise', 'favourite', 'impreza', 'levorg', 'carplay', 'brz', 'cvt', 'imprezas', 'nowarrived', 'wrx']


In [192]:
cleaned_data = data['Copy']
result = data['Clicks']

In [20]:
#split and correct
clean_d, res, datu = clean_dat(data,model)

145
145


In [21]:
#prints words from the dataset which is not present in google news vec
key2 = list()
for sentence in clean_d:
    for word in sentence.split(' '): 
        if word not in model:
            if word not in key2:
                key2.append(word)                
print(len(key2))
print(key2)

1
['favourite']


In [22]:
cleaned_data = clean_d
result = res

In [174]:
cleaned_data, result = remove_lang(clean_d, datu, model)

266
266


In [175]:
#prints words from the dataset which is not present in google news vec
key3 = list()
for sentence in cleaned_data:
    for word in sentence.split(' '): 
        if word not in model:
            if word not in key3:
                key3.append(word)
                
print(len(key3))
print(key3)

0
[]


In [23]:
len(cleaned_data)

145

In [24]:
len(result)

145

In [52]:
def RF_Regressor(X, y):
    #kfold
    kf = model_selection.KFold(n_splits=5) 
    kf.get_n_splits(X)
    #print(kf)
    #creating instance of RFRegressor 
    model1 = RandomForestRegressor(n_estimators=500,max_features='sqrt',n_jobs=-1,min_samples_leaf=60)#cross val split
    for train_index, test_index in kf.split(X):
        #print('TRAIN:', train_index, 'TEST:', test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]    
    #training
    model1.fit(X_train, y_train)
    #evaluating
    y_pred = model1.predict(X_test)
    #monotonic relationship as the relation between the variables is non linear
    spearman = spearmanr(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    print(f'Test data Spearman correlation: {spearman[0]:.3}')
    print(f'Test data pearson correlation: {pearson[0]:.3}')
    print('MSE')
    print(metrics.mean_squared_error(y_test, y_pred)) #mean square error
    print('R2')
    print(metrics.r2_score(y_test, y_pred)) #r2 score
    print('MAE')
    print(metrics.mean_absolute_error(y_test, y_pred)) #mae
    print('Variance Score')
    print(metrics.explained_variance_score(y_test, y_pred)) #mape

In [53]:
def ridge_regression(X, y):
    #kfold
    kf = model_selection.KFold(n_splits=5) 
    kf.get_n_splits(X)
    #print(kf)
    #cross val split with score and r2
    for train_index, test_index in kf.split(X):
    #print('TRAIN:', train_index, 'TEST:', test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    model2 = Ridge(alpha=0.5)
    # Fit the model
    model2.fit(X_train, y_train)
    y_pred = model2.predict(X_test)
    
    spearman = spearmanr(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    print(f'Test data Spearman correlation: {spearman[0]:.3}')
    print(f'Test data pearson correlation: {pearson[0]:.3}')
    print('MSE')
    print(metrics.mean_squared_error(y_test, y_pred)) #mean square error
    print('R2')
    print(metrics.r2_score(y_test, y_pred)) #r2 score
    print('MAE')
    print(metrics.mean_absolute_error(y_test, y_pred)) #mae
    print('Variance Score')
    print(metrics.explained_variance_score(y_test, y_pred)) 

In [195]:
X, y = bow_tfidf(cleaned_data, result)

(19118, 7112)
(19118,)


In [196]:
RF_Regressor(X, y)

Test data Spearman correlation: 0.107
Test data pearson correlation: 0.13
MSE
175.5643438940584
R2
0.0030529520362929663
MAE
2.312211835050464
Variance Score
0.007593508054656528


In [197]:
ridge_regression(X, y)

Test data Spearman correlation: 0.329
Test data pearson correlation: 0.182
MSE
241.9117751714945
R2
-0.37370279622617275
MAE
4.040666373203655
Variance Score
-0.37358883686906097


In [198]:
X, y = word2vec(cleaned_data, result)

(19118, 300)
(19118,)


In [199]:
RF_Regressor(X, y)

Test data Spearman correlation: 0.183
Test data pearson correlation: 0.182
MSE
170.40616252695142
R2
0.03234382951595838
MAE
2.69902181241004
Variance Score
0.032356003257481114


In [200]:
ridge_regression(X, y)

Test data Spearman correlation: 0.122
Test data pearson correlation: 0.189
MSE
172.43145100928444
R2
0.02084317209912434
MAE
3.6121474272703673
Variance Score
0.025413444333375446


In [201]:
X, y = g_uni_encoder(cleaned_data, result)

(19118, 512)
(19118,)


In [202]:
RF_Regressor(X, y)

Test data Spearman correlation: 0.186
Test data pearson correlation: 0.136
MSE
173.01166707070965
R2
0.017548398930558484
MAE
3.033783439136066
Variance Score
0.01844695496655413


In [203]:
ridge_regression(X, y)

Test data Spearman correlation: 0.227
Test data pearson correlation: 0.154
MSE
174.21428829626694
R2
0.01071928059112337
MAE
3.640063386800593
Variance Score
0.01793112795276053


In [204]:
X, y = infer_se(cleaned_data, result)

Found 3033(/7123) words with w2v vectors
Vocab size : 3033


/Users/rithika/Documents/247ai/summer-internship/models.py:222: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  sentences[stidx:stidx + bsize]), volatile=True)


nb sentences encoded : 19118
(19118, 4096)
(19118,)


In [205]:
RF_Regressor(X, y)

Test data Spearman correlation: 0.145
Test data pearson correlation: 0.201
MSE
170.32249493566226
R2
0.03281893827834059
MAE
3.3174003577720534
Variance Score
0.03745087650409262


In [206]:
ridge_regression(X, y)

Test data Spearman correlation: 0.221
Test data pearson correlation: 0.123
MSE
347.85919004692136
R2
-0.9753281613582871
MAE
6.779065207522452
Variance Score
-0.9703489877051856
